# Obtain MOR18-2 spectrum from factorization

In [10]:
import sys
import os
curdir = os.path.abspath(os.path.curdir)
sys.path.append(os.path.join(curdir,'FUImaging'))

In [11]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os, glob, json, pickle, csv
import scipy.stats
from collections import defaultdict
from PIL import Image


from regnmf import ImageAnalysisComponents as ia
from ModeSelector import AssignMode

### Specify parameter

In [12]:
stim_window = (3,5)

#method mask for components to be used. Use string with files to be opened. Use wildmark '*' for changing values.
method =  'nnmf_200_sm2_convex_negTimelowSP_sp*_ios_meas' #'sica_200_ios_meas' #

# datalocations [if you use the standard scheme, only modify basepath]
basepath = os.path.join('/media/jan/BackupWork/Documents/NewAnalysis')
data_location = os.path.join(basepath, 'MOBdecomposed')
bg_location = os.path.join(basepath, 'MOBconverted')
aliasfile = os.path.join(basepath, 'DataDicts', 'alias.json')
datadict_path = os.path.join(basepath, 'DataDicts')
savepath182modes = os.path.join(basepath, 'Vis', 'MOR18-2modes')
mor182spec_file = os.path.join(basepath, 'DataDicts', 'MOR18-2spec.json')
savepath182spec = os.path.join(basepath, 'Vis', 'MOR18-2spec')

In [13]:
animals = [a for a in os.listdir(data_location) if 'sph' not in a]

### GUI to manually assign MOR18-2 mode

Load or create dictionary with mode label (alias)

In [14]:
if os.path.exists(aliasfile):
    alias_dict = json.load(open(aliasfile))
else:
    alias_dict = {}
if method not in alias_dict:
    alias_dict[method]={}

Left panel shows all extracted modes (blue modes with high stimulus correlation >0.5, grey all others). Right panel shows currently selected mode. Press any pixel to select the mode, most strongly participation in this pixel. Hit button to mark current mode as MOR18-2 mode (becomes red-yellow)

In [18]:
for animal in np.sort(animals):
    bg = plt.imread(os.path.join(bg_location, animal, 'bg.png'))
    ts = ia.TimeSeries()

    filename = glob.glob(os.path.join(data_location, animal, method+'.npy'))
    assert len(filename)==1
    filename = filename[0].split('.')[0]
    ts.load(filename)
    
    odor_response = ia.TrialMean()(ia.CutOut(stim_window)(ts))
    ts.t2t = ia.CalcStimulusDrive()(odor_response)._series.squeeze() 
      
    if (animal in alias_dict[method]):
        alias_method = alias_dict[method][animal]
    else:
        alias_method = {}
    selector = AssignMode({'mf':ts, 'bg':bg, 'alias':alias_method})
    plt.show()
    if selector.alias:
        alias_dict[method][animal]=selector.alias
    elif animal in alias_dict[method]:
        alias_dict[method].pop(animal)

!!! warning: no repeated stimuli!!!


save the assignment

In [19]:
json.dump(alias_dict, open(aliasfile, 'w'))

### Summarize extracted modes for methods

In [15]:
for k, v in alias_dict.items():
    print('%s with %d identified MOR18-2 modes'%(k, len(v)))

nnmf_200_sm2_convex_neg_sp*_ios_meas with 48 identified MOR18-2 modes
nnmf_200_sm2_convex_negTime_sp*_ios_meas with 45 identified MOR18-2 modes
nnmf_200_sm2_convex_negTimelowSP_sp*_ios_meas with 51 identified MOR18-2 modes
sica_200_ios_meas with 44 identified MOR18-2 modes


### Plot all extracted MOR18-2 modes

In [21]:
methods = alias_dict.keys()

for method in methods:
    fig = plt.figure(figsize=(20, 3*len(alias_dict[method])))
    gs = matplotlib.gridspec.GridSpec(len(alias_dict[method]),1, left=0.02, hspace=0.7, top=0.97, right=0.99)
    
    modecolors = ['g','b','c']
    for ix, animal in enumerate(np.sort(alias_dict[method].keys())):
        gs_inner = matplotlib.gridspec.GridSpecFromSubplotSpec(1,2, subplot_spec=gs[ix], width_ratios=(2,8))
        ax = fig.add_subplot(gs_inner[0])    
        bg = Image.open(os.path.join(bg_location, animal, 'bg.png'))
        bg = bg.convert('L')
        
        mf = ia.TimeSeries()
        filename = glob.glob(os.path.join(data_location, animal, method+'.npy'))
        assert len(filename) == 1
        filename = filename[0].split('.')[0]
        mf.load(filename)
        
        bg = bg.resize(mf.base.shape[::-1])
        bg = np.asarray(bg)
        
        mf = ia.SingleSampleResponse()(ia.TrialMean()(ia.CutOut(stim_window)(mf)))
        order_ix = np.argsort(mf.label_stimuli)
        mf = ia.SelectTrials()(mf, order_ix)
        
        myextent = np.array([0, mf.base.shape[1], mf.base.shape[0], 0])-0.5
        ax.imshow(bg, interpolation='none', extent=myextent, cmap=plt.cm.bone)
        for ix, base_ix in enumerate(alias_dict[method][animal]['MOR18-2']):
            ax.contour(mf.base.shaped2D()[base_ix], [-0.3,0.3,0.7], colors=['m']+2*[modecolors[ix]], lw=2)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_ylabel(animal)
        
        ax = fig.add_subplot(gs_inner[1])
        for ix, base_ix in enumerate(alias_dict[method][animal]['MOR18-2']):
            ax.plot(mf.matrix_shaped()[:,base_ix], color=modecolors[ix])
        ax.set_xticks(range(len(order_ix)))
        ax.set_xticklabels(mf.label_stimuli, rotation='45', ha='right', size=10)
        ax.set_xlim((-0.5,len(order_ix)-0.5))
        ax.set_yticks([0,0.5])
        
    fig.savefig(os.path.join(savepath182modes, method+'.pdf'))
    plt.close('all')

### Collect and norm 18-2 response spectra 

In [16]:
norm_to = '554-12-1' #'554-12-1' for Methyl propionate, None for unormed responses
response_thres = 0.2 #exclude all modes where response to norm_to is below this [promille]
skip = ['110623s'] #animals to skip ('110623s' no undiluted odor)

Define function to clear stimuli labels. Thus it determines which stimuli are grouped together. Currently remove 'rechts'/'links' labels, as they correspond to syringe used. And '50grad' as there is no visible effect. Same with old label (marks an old sample of Acetaldehyde)

In [17]:
to_remove = ['rechts','links','re','li','50grad','old', 'o']

def clear_label(label):
    if '_' in label:
        label_split = label.split('_')
    for remove in set(to_remove).intersection(label_split):
        label_split.remove(remove)
    label = '_'.join(label_split)
    return label

calculate normed response spec

In [18]:
# load existing MOR18-2 spec
if os.path.exists(mor182spec_file):
    mor182spec = json.load(open(mor182spec_file))
else:
    mor182spec = {}

for method in alias_dict:
    print('=========== %s ============'%method)
    response_dict = defaultdict(list)
    for ix, animal in enumerate(np.sort(alias_dict[method].keys())):
        if animal in skip: 
            continue
        
        mf = ia.TimeSeries()
        filename = glob.glob(os.path.join(data_location, animal, method+'.npy'))
        assert len(filename) == 1
        filename = filename[0].split('.')[0]
        mf.load(filename)
      
        mf.label_stimuli = [clear_label(i) for i in mf.label_stimuli]
        mf = ia.SingleSampleResponse()(ia.TrialMean()(ia.CutOut(stim_window)(mf)))        
      
        for ix, base_ix in enumerate(alias_dict[method][animal]['MOR18-2']):
            resp = mf._series[:,base_ix]
            # normieren
            if norm_to:
                if resp[mf.label_stimuli.index(norm_to)]>response_thres:
                    resp /= resp[mf.label_stimuli.index(norm_to)]
                    [response_dict[k].append(v) for k,v in zip(mf.label_stimuli, resp)]
                else:
                    print('%s: mode %d (of %d) excluded'%(animal,ix,len(alias_dict[method][animal]['MOR18-2'])))
            else:
                [response_dict[k].append(v) for k,v in zip(mf.label_stimuli, resp)] 
    nameadd = 'normed' if norm_to else ''
    mor182spec[method+nameadd] = response_dict
json.dump(mor182spec, open(mor182spec_file, 'w'))

=========== nnmf_200_sm2_convex_neg_sp*_ios_meas ============
090331l: mode 0 (of 1) excluded
090331r: mode 0 (of 1) excluded
090414r: mode 0 (of 1) excluded
100212: mode 0 (of 1) excluded
100215: mode 1 (of 3) excluded
100225: mode 0 (of 1) excluded
110616b: mode 0 (of 1) excluded
120119: mode 0 (of 1) excluded
=========== nnmf_200_sm2_convex_negTime_sp*_ios_meas ============
090331l: mode 0 (of 1) excluded
100212: mode 0 (of 1) excluded
100215: mode 1 (of 3) excluded
100225: mode 0 (of 1) excluded
110616b: mode 0 (of 1) excluded
120119: mode 0 (of 2) excluded
120119: mode 1 (of 2) excluded
=========== nnmf_200_sm2_convex_negTimelowSP_sp*_ios_meas ============
090331l: mode 0 (of 1) excluded
090402l: mode 0 (of 1) excluded
090414l: mode 0 (of 2) excluded
090414l: mode 1 (of 2) excluded
090414r: mode 0 (of 1) excluded
090819: mode 0 (of 1) excluded
100212: mode 0 (of 1) excluded
100225: mode 0 (of 1) excluded
110518: mode 0 (of 1) excluded
110616b: mode 0 (of 1) excluded
110727a: mode 

## Plot spectra

In [27]:
method =  'nnmf_200_sm2_convex_negTimelowSP_sp*_ios_measnormed'
average_function = np.median

In [28]:
def condense_list_dict(dic, reducefct=np.mean):
    ''' apply reducefct to every value of dictionary'''
    reduced_dict = {}
    for k, v in dic.items():
        reduced_dict[k] = reducefct(v)
    return reduced_dict

#### Select relevant odors and sort according to avarage strength

In [29]:
#Select only pure odors. Hack! All mixtures contain either E or A 
pure_odors = [o for o in mor182spec[method].keys() if (len(o.split('_'))<2) and ('E' not in o) and ('A' not in o) and  ('B' not in o) ] 
pure_odor_resp = [average_function(mor182spec[method][o]) for o in pure_odors]
odors_sorted = [pure_odors[i] for i in np.argsort(pure_odor_resp)]
spec = [average_function(mor182spec[method][i]) for i in odors_sorted]

In [38]:
for i in np.where(p_val_argon<0.001)[0]:
 print(cas2name[odors_sorted[i]], spec[i])

Trimethyl phosphite -0.324269253216
3-Methylbut-2-enyl formate -0.173342433181
Anisole -0.132288523092
2-Methyl-2-pentenal -0.130215365022
Methyl isopropyl carbonate -0.08994051697
Benzaldehyde -0.0597079996151
Methyl formate 0.133754372053
2,3-Heptanedione 0.156884540954
amyl butyrate, mixture of isomers 0.16083703698
Isobutyltiglate 0.169596855629
Hexyl butyrate 0.173017600338
Isopropyl acetate 0.229456681463
Isobutyl propionate 0.230997435629
1-Ethylhexyl tiglate 0.267660181483
Allyl tiglate 0.268373165351
Ethyl acrylate 0.291821579071
Pyruvaldehyde 0.297462864492
Paraldehyde 0.309441537359
Ethyl formate 0.371104843884
Butyl acetate 0.500218760844
Propyl propionate 0.553256434215
Isobutyl acetate 0.583339530604
Allyl propionate 0.660015718688
Acetaldehyde 0.748555772007
Ethyl propionate 0.89593860875
Methyl acetate 0.971264294331
Propyl acetate 0.98025843292
Methyl propionate 1.0
Ethyl acetate 1.09808331797
Allyl acetate 1.13644434093


#### Calculate significance of response to Argon ('7440-37-1') and 2M2P ('623-36-9')

In [30]:
test = scipy.stats.mannwhitneyu#scipy.stats.ttest_ind #

p_val_argon = np.array([test(mor182spec[method]['7440-37-1'], mor182spec[method][i]) for i in odors_sorted])[:,1]
p_val_argon[np.isnan(p_val_argon)] = 1
p_val_argon[p_val_argon>0.05] = 1

p_val_2m2p = np.array([test(mor182spec[method]['623-36-9'], mor182spec[method][i]) for i in odors_sorted])[:,1]
p_val_2m2p[np.isnan(p_val_2m2p)] = 1
p_val_2m2p[p_val_2m2p>0.05] = 1

load odor names

In [31]:
cas2name = {l[0]:l[1] for l in csv.reader(open(os.path.join(datadict_path,'Name2MomCas.tab')),  delimiter='\t')}

#### Get heritage data of odors (how they where introduced into measurement set)

In [32]:
csvreader =  csv.reader(open(os.path.join(datadict_path, 'iterations.tab')),  delimiter='\t')
headings = csvreader.next()[2:]
cas2iteration = {l[0]:l[2:] for l in csvreader}


def first_entry_in_list(mylist, headings):
    for ix, entry in enumerate(mylist):
        if entry:
            return headings[ix]
    return 'train'

iteration = np.array([first_entry_in_list(cas2iteration[odor.strip()], headings) for odor in odors_sorted])

#### Create figure

In [33]:
fig = plt.figure(figsize=(7,7))
plot_heritage = [('elsewhere', 'y'), ('3rdIteration', 'c'), ('2ndIteration', 'g'), ('1stIteration','g'), ('train','b')]

gs = matplotlib.gridspec.GridSpec(5,1,bottom=0.88, top=0.97, left=0.15, right=0.95, hspace=0)
for ix, (key, color) in enumerate(plot_heritage):
    ax = fig.add_subplot(gs[ix])
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_ylabel(key, rotation=0, size=8, color=color, va='center')
    cmap_tmp = matplotlib.colors.LinearSegmentedColormap.from_list('tmp', ['w',color])
    plt.imshow((iteration==key).astype(float).reshape((1,-1)), interpolation='none', cmap=cmap_tmp, aspect='auto')

gs = matplotlib.gridspec.GridSpec(3,1, bottom=0.1, top=0.85, left=0.15, right=0.95, height_ratios=[1,1,20], hspace=0)

num_odors = len(spec)

ax = fig.add_subplot(gs[0])
ax.imshow(-np.log10(p_val_argon).reshape((1,-1)), plt.cm.binary, interpolation='none', aspect='auto', vmax=9.2)
ax.set_xlim((-0.5, num_odors-0.5))
ax.set_xticks([])
ax.set_yticks([])
ax.set_ylabel('p Argon', rotation='0', size=8)

ax = fig.add_subplot(gs[1])
ax.imshow(-np.log10(p_val_2m2p).reshape((1,-1)), plt.cm.binary, interpolation='none', aspect='auto', vmax=9.2)
ax.set_xlim((-0.5, num_odors-0.5))
ax.set_xticks([])
ax.set_yticks([])
ax.set_ylabel('p 2M2P', rotation='0', size=8)

ax = fig.add_subplot(gs[2])
# individual measurements
for ix, mol in enumerate(odors_sorted):
    ax.plot(ix, np.array(mor182spec[method][mol]).reshape((1,-1)), '.', mec='none', mfc='0.5')

#average activation
ax.step(np.arange(num_odors)+0.5, spec, 'b', lw=2)

#decorate
ax.set_xticks(np.arange(num_odors), minor=True)
ax.set_xticklabels([cas2name[m].decode("utf8") for m in odors_sorted], rotation='45', ha='right', size=6,minor=True)
#ax.set_xticks(np.arange(0,num_odors,len(spec)/4))
#ax.set_xticks([odors_sorted.index(i) for i in ['623-36-9', '7440-37-1',]], minor=True)
#ax.set_xticklabels(['2M2P','Argon'], minor=True, size=8, rotation='45', ha='right')

ax.set_xlim((-0.5, num_odors-0.5))
ax.set_yticks([0,0.5,1])
ax.grid()
ax.set_ylabel('rel. response')
ax.set_xlabel('odor #')

#fig.savefig(os.path.join(savepath182spec, method+'.pdf'))

plt.show()

In [26]:
plt.show()

### Compare sICA to NMF response

In [170]:
spec_sica = [average_function(mor182spec['sica_200_ios_meas'][i]) if i in mor182spec['sica_200_ios_meas'] else np.nan for i in odors_sorted ]
plt.plot(spec, spec_sica, 'x')
plt.plot([-0.5,1.5], [-0.5,1.5])
plt.show()

#### Distribution of sICA values for negative values in NMF

In [51]:
sica_nmf = np.array(spec_sica)[np.array(spec)<0]
sica_nmf = sica_nmf[np.logical_not(np.isnan(sica_nmf))]
plt.hist(sica_nmf, np.linspace(-1,1,51))
plt.show()